In [62]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from sqlalchemy import create_engine
from splinter import Browser

In [63]:
#splinter
browser = Browser('chrome', headless=False)
main_url='https://apps.oaklandca.gov/pec/Lobbyist_ReportsFiled.aspx'
browser.visit(main_url)

The chromedriver version (115.0.5790.170) detected in PATH at C:\utilities\chromedriver.exe might not be compatible with the detected chrome version (116.0.5845.188); currently, chromedriver 116.0.5845.96 is recommended for chrome 116.*, so it is advised to delete the driver in PATH and retry


In [64]:
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [65]:
records=[]
tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
links = [td.find('a')['href'] for td in tds if td.find('a')]

In [66]:
def navigation_has_loaded():
    if browser.is_element_present_by_css('.ListNavigation_Next', wait_time=.000001): # Check for next in case on first page
        return True
    elif browser.is_element_present_by_css('.ListNavigation_Previous', wait_time=.000001): # Check for previous in case on last page
        return True
    else: # Navigation has not loaded
        return False

In [67]:
navigation_has_loaded()

True

In [68]:
def contact_scrape():
    filing_html=browser.html
    filing_soup=BeautifulSoup(filing_html,'html.parser')
    tds=filing_soup.find_all('tr')
    for td in tds:
        # Check if this row contains 'Nothing reported...'
        if td.find(string="Nothing reported..."):
            continue  # Skip this iteration if found
            # Convert td elements to text and store them in the 'columns' list
        columns = [col.get_text(strip=True) for col in td]
        # You can add validation here to check if the length of columns is as expected
        # For example, if you expect each row to have 8 columns, you can check:
        if len(columns) == 8:
            records.append(columns)
        else:
            print(f"Skipped row with {len(columns)} columns: {columns}")

In [69]:
def lobbyist_scrape(arg):
  for link in arg:
      browser.visit('https://apps.oaklandca.gov/pec/'+link)
      if navigation_has_loaded():
        while browser.is_element_present_by_css('.ListNavigation_Previous', wait_time=.000001):
          try:
            previous_button=browser.find_by_css('.ListNavigation_Previous')
            previous_button.click()
          except Exception as e:
            print('Reset pages')
            break
        while navigation_has_loaded():
          contact_scrape()
          try:
            next_button = browser.find_by_css('.ListNavigation_Next')
            next_button.click()
            time.sleep(random.uniform(.5, 1))
          except Exception as e: #ends loop on last page
            print(f"Error or end of pages: {e}")
            break
      else:
         contact_scrape()
      if navigation_has_loaded():
        try:
          page_1_button = browser.find_by_xpath('//*[@id="wt133_OutSystemsUIWeb_wt2_block_wtContent_wtMainContent_OutSystemsUIWeb_wt223_block_wtContent_OutSystemsUIWeb_wtSchAAcc_block_wtContent_RichWidgets_wt5_block_wtPageNavigator_ctl00_wtLink8"]')
          page_1_button.click()
        except Exception as e:
          print(f"Error or end of pages: {e}")

In [70]:
#test
browser.visit(main_url)
records=[]
#lobbyist_scrape()

In [71]:
def test(arg):
    records=[]
    x=0
    while x<arg:
        if navigation_has_loaded():
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
            links = [td.find('a')['href'] for td in tds if td.find('a')]
            lobbyist_scrape()
            browser.visit(main_url)
            if navigation_has_loaded():
                try:
                    next_button = browser.find_by_css('.ListNavigation_Next')
                    next_button.click()
                    time.sleep(random.uniform(1, 1.1))
                except Exception as e: #ends loop on last page
                    print(f"Error or end of pages: {e}")
                    break
            else:
                print('bruh 2')
                break
        else:
            print('bruh')
            break
        x+=1

In [72]:
def full_scrape():
    while True:
        if navigation_has_loaded():
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            tds=soup.find_all('td',class_=['TableRecords_OddLine','TableRecords_EvenLine'])
            links=[]
            links = [td.find('a')['href'] for td in tds if td.find('a')]
            print(links)
            lobbyist_scrape(links)
            browser.visit(main_url)
            if navigation_has_loaded():
                try:
                    next_button = browser.find_by_css('.ListNavigation_Next')
                    next_button.click()
                    time.sleep(random.uniform(1, 1.1))
                except Exception as e: #ends loop on last page
                    print(f"Error or end of pages: {e}")
                    break
            else:
                print('bruh 2')
                break
        else:
            print('bruh')
            break


In [73]:
full_scrape()

['Lobbyist_Report.aspx?ReportId=35', 'Lobbyist_Report.aspx?ReportId=36', 'Lobbyist_Report.aspx?ReportId=37', 'Lobbyist_Report.aspx?ReportId=38', 'Lobbyist_Report.aspx?ReportId=39', 'Lobbyist_Report.aspx?ReportId=40', 'Lobbyist_Report.aspx?ReportId=41', 'Lobbyist_Report.aspx?ReportId=42', 'Lobbyist_Report.aspx?ReportId=43', 'Lobbyist_Report.aspx?ReportId=44', 'Lobbyist_Report.aspx?ReportId=45', 'Lobbyist_Report.aspx?ReportId=46', 'Lobbyist_Report.aspx?ReportId=47', 'Lobbyist_Report.aspx?ReportId=49', 'Lobbyist_Report.aspx?ReportId=50']
Skipped row with 7 columns: ['', 'Organization', 'Name', 'Address', 'Has Activity', 'Amount', '']
Skipped row with 7 columns: ['', 'AT&T', 'AT&T', '430 Bush St., 5th Floor', 'No', 'Not Stated', '']
Skipped row with 7 columns: ['', 'Ellwood Commercial Real Estate', 'Patrick Ellwood', '1354 Piedmont Ave.', 'No', 'Not Stated', '']
Skipped row with 7 columns: ['', 'Falck Ambulance Northern California', 'Carolina Snypes', '2190 S. McDowell', 'No', 'Not Stated'

In [77]:
contact_df=pd.DataFrame(records[1:],columns=records[0])
#contact_df.drop_duplicates(inplace=True, ignore_index=False)
contact_df=contact_df[['Client','Municipal Decision','Subject','Position','Narrative','City Official']]
contact_df=contact_df[contact_df.Client != 'Client']
contact_df=contact_df.reset_index(drop=True)
contact_df.to_csv('contact_df.csv')

In [79]:
contact_csv=pd.read_csv('contact_df.csv',index_col=0)
# Create an SQLite database in memory
engine = create_engine('sqlite:///my_database.db')


In [80]:

# Write the dataframe to a new SQL table
contact_csv.to_sql('contact_table', engine, if_exists='replace', index=False)
contact_csv.to_sql('contact_df.sql',con=engine)

3254

In [85]:
contact_csv.head(60)

,Client,Municipal Decision,Subject,Position,Narrative,City Official
0,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,"Rebecca Kaplan, Council Member, At-Large"
1,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,"Dan Kalb, Council Member, District 1"
2,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,"Sheng Thao, Council Member, District 4"
3,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,"Noel Gallo, Council Member, District 5"
4,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,"Libby Schaaf, Mayor"
5,"Lyft, Inc.",Public Contract,Transportation,Support,Supporting updates to the bike share franchise...,Department of Transportation - Director of Tra...
6,Rubicon Point Partners,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1750 Broadway use permit proposal.,"Rebecca Kaplan, Council Member, At-Large"
7,Rubicon Point Partners,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1750 Broadway use permit proposal.,"Dan Kalb, Council Member, District 1"
8,Rubicon Point Partners,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1750 Broadway use permit proposal.,"Nikki Fortunato Bas, Council Member, District 2"
9,Rubicon Point Partners,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1750 Broadway use permit proposal.,"Lynette Gibson McElhaney, Council Member, Dist..."


In [82]:
contact_csv.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,3254,3254,3254,3254,3249,3253
unique,120,5,23,4,504,165
top,Oakland Athletics,"Land Use (Permit, Zoning, Study)",Other,Support,Support the development of the new ballpark an...,"Nikki Fortunato Bas, Council Member, District 2"
freq,842,1751,1254,1734,501,179


In [84]:
df_with_missing_values = contact_csv[contact_csv.isnull().any(axis=1)]
df_with_missing_values

,Client,Municipal Decision,Subject,Position,Narrative,City Official
16,Rubicon Point Partners,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1750 Broadway use permit proposal.,NaN
1634,Larkin Street Youth Services,Information only,Social Services,Informational meeting,NaN,Office of the Mayor - Chief of Staff
1635,DoorDash,Information only,Technology,Informational meeting,NaN,"Loren Taylor, Council Member, District 6"
1636,DoorDash,Information only,Technology,Informational meeting,NaN,"Treva Reid, Council Member, District 7"
1637,DoorDash,Information only,Technology,Informational meeting,NaN,"Dan Kalb, Council Member, District 1"
1638,DoorDash,Information only,Technology,Informational meeting,NaN,"Nikki Fortunato Bas, Council Member, District 2"


In [88]:
x='Oakland Athletics'
df_with_specific_value = contact_csv[contact_csv['Client'] == x]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,842,842,842,842,842,842
unique,1,1,1,1,8,75
top,Oakland Athletics,"Land Use (Permit, Zoning, Study)",Other,Support,Support the development of the new ballpark an...,"Nikki Fortunato Bas, Council Member, District 2"
freq,842,842,842,842,501,22


In [118]:
municple_decisions=contact_csv['Municipal Decision'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Municipal Decision'] == municple_decisions[0]]
df_with_specific_value.describe()


,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,93,93,93,93,93,93
unique,9,1,9,3,28,29
top,Marshall Retail Group,Public Contract,Airport,Support,Bike share franchise agreement,"Andreas Cluver, Board of Port"
freq,35,93,43,75,19,9


In [119]:
municple_decisions=contact_csv['Municipal Decision'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Municipal Decision'] == municple_decisions[1]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,1751,1751,1751,1751,1751,1750
unique,46,1,10,4,141,115
top,Oakland Athletics,"Land Use (Permit, Zoning, Study)",Other,Support,Support the development of the new ballpark an...,"Dan Kalb, Council Member, District 1"
freq,842,1751,1117,1228,501,94


In [120]:
municple_decisions=contact_csv['Municipal Decision'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Municipal Decision'] == municple_decisions[2]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,970,970,970,970,970,970
unique,49,1,19,4,206,106
top,Earthjustice,"Ordinance, Policy, or Other Legislative Matter",Environment,Policy development,Advocating to transition the Port of Oakland t...,"Nikki Fortunato Bas, Council Member, District 2"
freq,314,970,395,522,303,67


In [121]:
municple_decisions=contact_csv['Municipal Decision'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Municipal Decision'] == municple_decisions[3]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,123,123,123,123,123,123
unique,27,1,13,4,51,45
top,Verizon,"Other Permits, Administrative Decisions",Technology,Support,Discuss the installation of Informational Digi...,Department of Transportation - Assistant Direc...
freq,26,123,48,73,11,9


In [122]:
municple_decisions=contact_csv['Municipal Decision'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Municipal Decision'] == municple_decisions[4]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,317,317,317,317,312,317
unique,45,1,17,3,90,82
top,PricewaterhouseCoopers LLP,Information only,Housing/Property Tax,Informational meeting,CEO Action for Racial Equity efforts relating ...,"Nikki Fortunato Bas, Council Member, District 2"
freq,152,317,144,148,25,23


In [125]:
Subjects=contact_csv['Subject'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Subject'] == Subjects[0]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,189,189,189,189,189,189
unique,17,5,1,4,39,34
top,Insight Terminal Solutions,"Land Use (Permit, Zoning, Study)",Transportation,Informational meeting,potential no-coal options at terminal,"Rebecca Kaplan, Council Member, At-Large"
freq,71,71,189,94,25,17


In [126]:
Subjects=contact_csv['Subject'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Subject'] == Subjects[1]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,408,408,408,408,408,407
unique,31,4,1,4,95,77
top,The Michaels Organization,"Land Use (Permit, Zoning, Study)",Planning and Building Permits,Support,1396 5th Street Use Permit,"Dan Kalb, Council Member, District 1"
freq,122,388,408,267,39,36


In [127]:
Subjects=contact_csv['Subject'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Subject'] == Subjects[2]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,1254,1254,1254,1254,1254,1254
unique,25,5,1,4,59,92
top,Oakland Athletics,"Land Use (Permit, Zoning, Study)",Other,Support,Support the development of the new ballpark an...,"Nikki Fortunato Bas, Council Member, District 2"
freq,842,1117,1254,900,501,54


In [128]:
Subjects=contact_csv['Subject'].unique().tolist()
df_with_specific_value = contact_csv[contact_csv['Subject'] == Subjects[3]]
df_with_specific_value.describe()

,Client,Municipal Decision,Subject,Position,Narrative,City Official
count,35,35,35,35,35,35
unique,8,3,1,3,14,16
top,Kaiser Permanente,"Ordinance, Policy, or Other Legislative Matter",Health,Policy development,Item 20-759 - Wanted Council to involve Public...,"Sheng Thao, Council Member, District 4"
freq,19,26,35,23,8,5
